In [ ]:
pip install torch torchvision matplotlib

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [ ]:
# Load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

In [4]:
# Crate data loaders

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Build a simple CNN model using torch.nn

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Calculate the correct flattened size
        self.fc1 = nn.Linear(32 * 13 * 13, 128)  # Adjust dimensions to match the output of convolutional layers
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        # Flatten to the correct size
        x = x.view(-1, 32 * 13 * 13)  # Flatten to match fc1 input
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Train the Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):  # Number of epochs
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Test the model

In [ ]:
correct = 0
total = 0
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


Save the model

In [9]:
torch.save(model.state_dict(), 'mnist_cnn.pth')


Visualize Predictions

In [ ]:
import matplotlib.pyplot as plt

model.eval()
sample_data, sample_label = next(iter(test_loader))
sample_data = sample_data.to(device)

outputs = model(sample_data)
_, predictions = torch.max(outputs, 1)

for i in range(5):
    plt.imshow(sample_data[i].cpu().squeeze(), cmap='gray')
    plt.title(f"Prediction: {predictions[i].item()}")
    plt.show()
